# Interactive Visualization Lab

Complete the following set of exercises to solidify your knowledge of interactive visualization using Plotly, Cufflinks, and IPyWidgets.

In [1]:
import pandas as pd
import plotly.plotly as py
import cufflinks as cf
from ipywidgets import interact
from datetime import datetime, date
from ipywidgets import interact, widgets
import re
cf.go_offline()

In [2]:
data = pd.read_excel('../data/Online Retail.xlsx')

## 1. Create an interactive bar chart showing total quantity and revenue by country (excluding United Kingdom) for the month of April 2011.

In [59]:
mask = (~(data['Country'] == 'United Kingdom'))
mask1 = (data.InvoiceDate.dt.year == 2011)
mask2 = (data.InvoiceDate.dt.month == 4)        
data.loc[mask & mask1 & mask2,:][['Country','Quantity','Revenue']].iplot(kind='bar', x='Country')

## 2. Create an interactive line chart showing quantity and revenue sold to France between January 1st and May 31st 2011.

In [108]:
mask = (data['Country'] == 'France')
mask1 = (data.InvoiceDate.dt.date >=  date(2011,1,1))
mask2 = (data.InvoiceDate.dt.date <= date(2011,5,31))
df2 = data.loc[mask & mask1 & mask2,:][['InvoiceDate','Quantity','Revenue']].groupby('InvoiceDate').sum().reset_index()
df2['date'] = df2.InvoiceDate.dt.date
df2.drop(columns = 'InvoiceDate', inplace=True)
df2 = df2.groupby('date').sum().reset_index()
df2.iplot(kind='line', x='date')

## 3. Create an interactive scatter plot showing the relationship between average quantity (x-axis) and average unit price (y-axis) for the product PARTY BUNTING with the plot points color-coded by country (categories).

In [127]:
mask = data.Description == 'PARTY BUNTING'
df =data.loc[mask,:].groupby(by='Country').mean().reset_index()[['Country','Quantity','UnitPrice']]
df.iplot(kind='scatter',x='Quantity',y='UnitPrice',categories='Country')

C:\Users\pedro\Anaconda3\lib\site-packages\cufflinks\plotlytools.py:805: FutureWarning:

The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead

C:\Users\pedro\Anaconda3\lib\site-packages\cufflinks\plotlytools.py:808: FutureWarning:

The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead



## 4. Create a set of interactive histograms showing the distributions of quantity per invoice for the following countries: EIRE, Germany, France, and Netherlands.

In [140]:
@interact(country= ['EIRE', 'Germany', 'France', 'Netherlands'])
def plot_grah(country):
    data.query(f'Country == "{country}"')[['Quantity']].iplot(kind='hist',histfunc='count')

interactive(children=(Dropdown(description='country', options=('EIRE', 'Germany', 'France', 'Netherlands'), va…

## 5. Create an interactive side-by-side bar chart showing the revenue by country listed below (bars) for each of the products listed below.

In [20]:
product_list = ['JUMBO BAG RED RETROSPOT', 
                'CREAM HANGING HEART T-LIGHT HOLDER',
                'REGENCY CAKESTAND 3 TIER']

country_list = ['EIRE', 'Germany', 'France', 'Netherlands']

In [21]:
@interact(product=product_list, country=country_list)
def get_plote(product,country):
    data.query(f'Country == "{country}" & Description == "{product}"').iplot(kind='bar', x='InvoiceDate', y='Revenue', histfunc=sum)

interactive(children=(Dropdown(description='product', options=('JUMBO BAG RED RETROSPOT', 'CREAM HANGING HEART…

## 6. Create an interactive line chart showing quantity sold by day for the United Kingdom. Add drop-down boxes for Year and Month that allow you to filter the date range that appears in the chart.

In [42]:
uk = data[data['Country']=='United Kingdom']
uk['date'] = uk['InvoiceDate'].dt.date
year_lst = list(uk['InvoiceDate'].dt.year.unique())
month_lst = list(uk['InvoiceDate'].dt.month.unique())

In [49]:
@interact(year=year_lst, month=month_lst)
def get_plot_uk(year,month):
    df_temp = uk.query(f'InvoiceDate.dt.year == {year} & InvoiceDate.dt.month == {month}').groupby('date').sum().reset_index()
    df_temp.iplot(kind='line', x='date', y='Revenue', histfunc='sum')

interactive(children=(Dropdown(description='year', options=(2010, 2011), value=2010), Dropdown(description='mo…

## 7. Create an interactive scatter plot that plots number of invoices (x-axis) vs. number of customers (y-axis) and the plot points represent individual products. Add two sliders that control the x and y axis ranges.

In [165]:
agg_func = {'InvoiceNo':'nunique',
            'Quantity':'sum',
            'UnitPrice':'mean',
            'Revenue':'sum',
            'CustomerID':'nunique'}

products = uk.groupby('Description').agg(agg_func).reset_index()
qtd_lst = list(set(products['Quantity'].sort_values()))
cst_lst = list(set(products['CustomerID'].sort_values()))

In [172]:
qtd = widgets.SelectionSlider(options=qtd_lst)
cust = widgets.SelectionSlider(options=cst_lst)
@interact(quantity=qtd, n_customers=cust)
def get_plot(quantity, n_customers):
    df_temp = products.query(f'Quantity == {quantity} & CustomerID == {n_customers}')
    df_temp.iplot(kind= 'scatter', x='Quantity', y='CustomerID', categories='Description')

interactive(children=(SelectionSlider(description='quantity', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, …

## 8. Creat an interactive bar chart that shows revenue by product description. Add a text field widget that filters the results to show the product that contain the text entered in their description.

In [4]:
def find_products(product):
    products = data['Description'].unique()
    return [item for item in products if len(re.findall(f'.*{product.upper()}.*', item))>0]

In [6]:
@interact(product='')
def get_plot(product):
    lst_result = find_products(product)
    if len(lst_result) > 0:
        mask = (data['Description'].isin(lst_result))
        data.loc[mask,:].groupby('Description')['Revenue'].sum().iplot(kind='bar')
    else:
        print(f'Not found product contain text {product}')

interactive(children=(Text(value='', description='product'), Output()), _dom_classes=('widget-interact',))